In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
%load_ext autoreload

%autoreload 2

from IPython.display import display
import pandas as pd
pd.options.display.max_columns = None
from IPython.display import display, HTML
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
import gc

In [2]:
train_identity=pd.read_csv('train_identity.csv')
train_transaction=pd.read_csv('train_transaction.csv')
sample_submission=pd.read_csv('sample_submission.csv')
test_identity=pd.read_csv('test_identity.csv')
test_transaction=pd.read_csv('test_transaction.csv')

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

In [37]:
del train_identity,train_transaction,test_identity,test_transaction

In [12]:
START_DATE = '2017-04-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
train['TransactionDT_new'] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))

In [15]:
START_DATE_test= '2017-09-30'
startdatex = datetime.datetime.strptime(START_DATE_test, '%Y-%m-%d')
test['TransactionDT_new'] = test['TransactionDT'].apply(lambda x: (startdatex + datetime.timedelta(seconds = x)))

In [34]:
train['Month']= pd.to_datetime(train['TransactionDT_new']).dt.month
train['Day']= pd.to_datetime(train['TransactionDT_new']).dt.day
train['Hour']= pd.to_datetime(train['TransactionDT_new']).dt.hour
train['Minute']= pd.to_datetime(train['TransactionDT_new']).dt.minute

In [36]:
test['Month']= pd.to_datetime(test['TransactionDT_new']).dt.month
test['Day']= pd.to_datetime(test['TransactionDT_new']).dt.day
test['Hour']= pd.to_datetime(test['TransactionDT_new']).dt.hour
test['Minute']= pd.to_datetime(test['TransactionDT_new']).dt.minute

In [39]:
train=train.drop('TransactionDT_new',axis=1)
test=test.drop('TransactionDT_new',axis=1)

In [41]:
#This is a function that downcast the integer columns
def downcast_df_int_columns(df):
    list_of_columns = list(df.select_dtypes(include=["int32", "int64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting integers for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="integer")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [42]:
#This is a function that downcast the float columns, if you have too many columns to adjust and do not want to see to many messages proceesing, you could comment our the print() columns
def downcast_df_float_columns(df):
    list_of_columns = list(df.select_dtypes(include=["float64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting float for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="float")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [6]:
train.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 435 entries, TransactionID_x to DeviceInfo
dtypes: float64(400), int64(4), object(31)
memory usage: 2.5 GB


In [6]:
test.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 434 entries, TransactionID_x to DeviceInfo
dtypes: float64(400), int64(3), object(31)
memory usage: 2.2 GB


In [43]:
downcast_df_int_columns(train)
downcast_df_float_columns(train)
downcast_df_int_columns(test)
downcast_df_float_columns(test)

downcasting integers for: ['TransactionID_x', 'isFraud', 'TransactionDT', 'card1', 'Month', 'Day', 'Hour', 'Minute'] 

reduced memory usage for:   TransactionID_x   from     4.72 to     2.36
reduced memory usage for:   isFraud           from     4.72 to     0.59
reduced memory usage for:   TransactionDT     from     4.72 to     2.36
reduced memory usage for:   card1             from     4.72 to     1.18
reduced memory usage for:   Month             from     4.72 to     0.59
reduced memory usage for:   Day               from     4.72 to     0.59
reduced memory usage for:   Hour              from     4.72 to     0.59
reduced memory usage for:   Minute            from     4.72 to     0.59
done
downcasting float for: ['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2'

reduced memory usage for:   V24               from     4.72 to     2.36
reduced memory usage for:   V25               from     4.72 to     2.36
reduced memory usage for:   V26               from     4.72 to     2.36
reduced memory usage for:   V27               from     4.72 to     2.36
reduced memory usage for:   V28               from     4.72 to     2.36
reduced memory usage for:   V29               from     4.72 to     2.36
reduced memory usage for:   V30               from     4.72 to     2.36
reduced memory usage for:   V31               from     4.72 to     2.36
reduced memory usage for:   V32               from     4.72 to     2.36
reduced memory usage for:   V33               from     4.72 to     2.36
reduced memory usage for:   V34               from     4.72 to     2.36
reduced memory usage for:   V35               from     4.72 to     2.36
reduced memory usage for:   V36               from     4.72 to     2.36
reduced memory usage for:   V37               from     4.72 to  

reduced memory usage for:   V137              from     4.72 to     2.36
reduced memory usage for:   V138              from     4.72 to     2.36
reduced memory usage for:   V139              from     4.72 to     2.36
reduced memory usage for:   V140              from     4.72 to     2.36
reduced memory usage for:   V141              from     4.72 to     2.36
reduced memory usage for:   V142              from     4.72 to     2.36
reduced memory usage for:   V143              from     4.72 to     2.36
reduced memory usage for:   V144              from     4.72 to     2.36
reduced memory usage for:   V145              from     4.72 to     2.36
reduced memory usage for:   V146              from     4.72 to     2.36
reduced memory usage for:   V147              from     4.72 to     2.36
reduced memory usage for:   V148              from     4.72 to     2.36
reduced memory usage for:   V149              from     4.72 to     2.36
reduced memory usage for:   V150              from     4.72 to  

reduced memory usage for:   V250              from     4.72 to     2.36
reduced memory usage for:   V251              from     4.72 to     2.36
reduced memory usage for:   V252              from     4.72 to     2.36
reduced memory usage for:   V253              from     4.72 to     2.36
reduced memory usage for:   V254              from     4.72 to     2.36
reduced memory usage for:   V255              from     4.72 to     2.36
reduced memory usage for:   V256              from     4.72 to     2.36
reduced memory usage for:   V257              from     4.72 to     2.36
reduced memory usage for:   V258              from     4.72 to     2.36
reduced memory usage for:   V259              from     4.72 to     2.36
reduced memory usage for:   V260              from     4.72 to     2.36
reduced memory usage for:   V261              from     4.72 to     2.36
reduced memory usage for:   V262              from     4.72 to     2.36
reduced memory usage for:   V263              from     4.72 to  

done
downcasting integers for: ['TransactionID_x', 'TransactionDT', 'card1', 'Month', 'Day', 'Hour', 'Minute'] 

reduced memory usage for:   TransactionID_x   from     4.05 to     2.03
reduced memory usage for:   TransactionDT     from     4.05 to     2.03
reduced memory usage for:   card1             from     4.05 to     1.01
reduced memory usage for:   Month             from     4.05 to     0.51
reduced memory usage for:   Day               from     4.05 to     0.51
reduced memory usage for:   Hour              from     4.05 to     0.51
reduced memory usage for:   Minute            from     4.05 to     0.51
done
downcasting float for: ['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14',

reduced memory usage for:   V25               from     4.05 to     2.03
reduced memory usage for:   V26               from     4.05 to     2.03
reduced memory usage for:   V27               from     4.05 to     2.03
reduced memory usage for:   V28               from     4.05 to     2.03
reduced memory usage for:   V29               from     4.05 to     2.03
reduced memory usage for:   V30               from     4.05 to     2.03
reduced memory usage for:   V31               from     4.05 to     2.03
reduced memory usage for:   V32               from     4.05 to     2.03
reduced memory usage for:   V33               from     4.05 to     2.03
reduced memory usage for:   V34               from     4.05 to     2.03
reduced memory usage for:   V35               from     4.05 to     2.03
reduced memory usage for:   V36               from     4.05 to     2.03
reduced memory usage for:   V37               from     4.05 to     2.03
reduced memory usage for:   V38               from     4.05 to  

reduced memory usage for:   V138              from     4.05 to     2.03
reduced memory usage for:   V139              from     4.05 to     2.03
reduced memory usage for:   V140              from     4.05 to     2.03
reduced memory usage for:   V141              from     4.05 to     2.03
reduced memory usage for:   V142              from     4.05 to     2.03
reduced memory usage for:   V143              from     4.05 to     2.03
reduced memory usage for:   V144              from     4.05 to     2.03
reduced memory usage for:   V145              from     4.05 to     2.03
reduced memory usage for:   V146              from     4.05 to     2.03
reduced memory usage for:   V147              from     4.05 to     2.03
reduced memory usage for:   V148              from     4.05 to     2.03
reduced memory usage for:   V149              from     4.05 to     2.03
reduced memory usage for:   V150              from     4.05 to     2.03
reduced memory usage for:   V151              from     4.05 to  

reduced memory usage for:   V251              from     4.05 to     2.03
reduced memory usage for:   V252              from     4.05 to     2.03
reduced memory usage for:   V253              from     4.05 to     2.03
reduced memory usage for:   V254              from     4.05 to     2.03
reduced memory usage for:   V255              from     4.05 to     2.03
reduced memory usage for:   V256              from     4.05 to     2.03
reduced memory usage for:   V257              from     4.05 to     2.03
reduced memory usage for:   V258              from     4.05 to     2.03
reduced memory usage for:   V259              from     4.05 to     2.03
reduced memory usage for:   V260              from     4.05 to     2.03
reduced memory usage for:   V261              from     4.05 to     2.03
reduced memory usage for:   V262              from     4.05 to     2.03
reduced memory usage for:   V263              from     4.05 to     2.03
reduced memory usage for:   V264              from     4.05 to  

no columns to downcast
done
no columns to downcast
done


In [11]:
pd.set_option('display.max_columns', None)

train.head()

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,TransactionID_y,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2987004.0,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,29870

In [46]:
#This is a function that convert a list of columns from object to categorical
def convert_columns_to_catg(df, column_list):
    for col in column_list:
        print("converting", col.ljust(30), "size: ", round(df[col].memory_usage(deep=True)*1e-6,2), end="\t")
        df[col] = df[col].astype("category")
        print("->\t", round(df[col].memory_usage(deep=True)*1e-6,2))

In [47]:
convert_columns_to_catg(train, ['id_12', 'id_13','id_14','id_15','id_17','id_16','id_18','id_19','id_20',
                                'id_21','id_22','id_23','id_24','id_25','id_26','id_27','id_28','id_29','id_32',
                                'id_30', 'id_31', 'id_33', 'id_34','id_35', 'id_36', 'id_37', 'id_38', 
                                'DeviceType', 'DeviceInfo','ProductCD', 'card1', 'card2', 
                                'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 
                                'M1', 'M2','M3','M4','M5','M6','M7','M8','M9'])

converting id_12                          size:  23.59	->	 0.59
converting id_13                          size:  2.36	->	 0.59
converting id_14                          size:  2.36	->	 0.59
converting id_15                          size:  23.03	->	 0.59
converting id_17                          size:  2.36	->	 0.6
converting id_16                          size:  22.97	->	 0.59
converting id_18                          size:  2.36	->	 0.59
converting id_19                          size:  2.36	->	 1.21
converting id_20                          size:  2.36	->	 1.19
converting id_21                          size:  2.36	->	 1.21
converting id_22                          size:  2.36	->	 0.59
converting id_23                          size:  19.12	->	 0.59
converting id_24                          size:  2.36	->	 0.59
converting id_25                          size:  2.36	->	 1.19
converting id_26                          size:  2.36	->	 0.59
converting id_27                          size:  19.

In [48]:
convert_columns_to_catg(test, ['id_12', 'id_13','id_14','id_15','id_17','id_16','id_18','id_19','id_20',
                                'id_21','id_22','id_23','id_24','id_25','id_26','id_27','id_28','id_29','id_32',
                                'id_30', 'id_31', 'id_33', 'id_34','id_35', 'id_36', 'id_37', 'id_38', 
                                'DeviceType', 'DeviceInfo','ProductCD', 'card1', 'card2', 
                                'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 
                                'M1', 'M2','M3','M4','M5','M6','M7','M8','M9'])

converting id_12                          size:  20.83	->	 0.51
converting id_13                          size:  2.03	->	 0.51
converting id_14                          size:  2.03	->	 0.51
converting id_15                          size:  20.23	->	 0.51
converting id_17                          size:  2.03	->	 0.51
converting id_16                          size:  20.16	->	 0.51
converting id_18                          size:  2.03	->	 0.51
converting id_19                          size:  2.03	->	 1.04
converting id_20                          size:  2.03	->	 1.04
converting id_21                          size:  2.03	->	 1.04
converting id_22                          size:  2.03	->	 0.51
converting id_23                          size:  16.44	->	 0.51
converting id_24                          size:  2.03	->	 0.51
converting id_25                          size:  2.03	->	 1.03
converting id_26                          size:  2.03	->	 0.51
converting id_27                          size:  16

In [10]:
train.groupby('DeviceInfo').agg({'TransactionDT':'min'}).sort_values('TransactionDT').head(20)

,TransactionDT
DeviceInfo,
SAMSUNG SM-G892A Build/NRD90M,86400
iOS Device,86401
Windows,86469
MacOS,86506
SM-G930V Build/NRD90M,86555
BLADE A602 Build/MRA58K,86618
XT1635-02 Build/NPN26.118-22-2,86808
Z970,87202
SM-N920V Build/NRD90M,87319


In [15]:
train.memory_usage().sum()

945897984

In [49]:
test.memory_usage().sum()

813271129

In [17]:
train

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.500,W,13926,321.0,150.0,discover,142.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,...,chrome 62.0,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,...,chrome 62.0,32.0,1920x1080,match_status:2,T,F,T,T,mobile,NaN
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows


In [20]:
columns=['card2','card3','card4','card5','card6','addr2','D1',
      'V14','V15','V16','V17','V18','V19','V20','V21','V22',
     'V23','V24','V25','V26','V27','V28','V95','V96','V97','V98','V99','V100',
     'V101','V102','V103','V104','V105','V106','V107','V108','V109','V110',
     'V111','V112','V113','V114','V115','V116','V117','V118','V119','V120','V121','V122','V123'
     ,'V124','V125','V126','V127','V128','V129','V130','V131','V132','V133','V134','V135','V136','V137',
     'V281','V282','V284','V286','V287','V288','V289','V290','V291','V292','V293'
     ,'V294','V295','V296','V297','V298','V299','V300','V301','V305','V306'
     ,'V309','V310','V311','V312','V313','V314','V316','V317','V318','V319','V320','V321']

In [21]:
for i in columns:
    train[i]=train[i].fillna(train[i].mode()[0])

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled

filled



In [22]:
for i in columns:
    test[i]=test[i].fillna(test[i].mode()[0])

In [36]:
#print('Memory usage train:', round(train_transaction.memory_usage(deep=True).sum()/1024/1024, 2), 'MB')

Memory usage train: 854.58 MB


In [ ]:
#all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

In [50]:
def encodeLabels(X_train, X_test):
    # Label Encoding
    for f in X_train.columns:
        if X_train[f].dtype=='object' or X_test[f].dtype=='object':
            lbl = LabelEncoder()
            lbl.fit(list(X_train[f].values) + list(X_test[f].values))
            X_train[f] = lbl.transform(list(X_train[f].values))
            X_test[f] = lbl.transform(list(X_test[f].values))
    return X_train, X_test

def readData(path, encode=True):

    X_train = train.drop('isFraud', axis=1)
    y_train = train['isFraud'].copy()
    X_test = test.copy()
    y_test = sample_submission.copy()

    if encode==True:
        X_train, X_test = encodeLabels(X_train, X_test)

    return X_train, y_train, X_test, y_test

In [51]:
X, y, test, submission = readData("../input/")

In [52]:
nsplits = 5
submission["isFraud"] = 0
skf = StratifiedKFold(n_splits=nsplits, shuffle=True, random_state=0)

parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'n_estimators': 500,
    'max_depth': 9,
    'bagging_fraction': 0.9,
    'feature_fraction': 0.9,
}

for idx_train, idx_test in skf.split(X, y):
    train_data = lgb.Dataset(data=X.iloc[idx_train], label=y.iloc[idx_train])
    valid_data = lgb.Dataset(data=X.iloc[idx_test], label=y.iloc[idx_test])
    model = lgb.train(params=parameters, train_set=train_data, valid_sets=valid_data, \
                      verbose_eval=500, early_stopping_rounds=100)
    submission['isFraud'] = submission['isFraud'] + model.predict(test)
    
submission['isFraud'] = submission['isFraud'] / 5

Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.963699
Did not meet early stopping. Best iteration is:
[500]	valid_0's auc: 0.963699
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.963314
Did not meet early stopping. Best iteration is:
[500]	valid_0's auc: 0.963314
Training until validation scores don't improve for 100 rounds.
[500]	valid_0's auc: 0.959906
Did not meet early stopping. Best iteration is:
[500]	valid_0's auc: 0.959906


MemoryError: 

In [53]:
submission

,TransactionID,isFraud
0,3663549,0.000689
1,3663550,0.010408
2,3663551,0.002602
3,3663552,0.000804
4,3663553,0.007347
5,3663554,0.004516
6,3663555,0.018378
7,3663556,0.088969
8,3663557,0.004969
9,3663558,0.015666


In [54]:
submission.to_csv('submission_3.csv')

In [55]:
submission1=pd.read_csv('submission.csv')
submission2=pd.read_csv('submission_2.csv')
submission3=pd.read_csv('submission_3.csv')

In [56]:
submission5=sample_submission
submission5['isFraud']=0.5*submission1['isFraud']+submission2['isFraud']*0.25+submission3['isFraud']*0.25

In [57]:
submission5.to_csv('submission0_5.csv')

In [3]:
train_transaction

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
(pd.DataFrame(train_transaction['card1'].value_counts())['card1']==1).sum()

3444

In [37]:
null_columns=train_transaction.isnull().sum()

In [38]:
null_columns

TransactionID          0
isFraud                0
TransactionDT          0
TransactionAmt         0
ProductCD              0
card1                  0
card2               8933
card3               1565
card4               1577
card5               4259
card6               1571
addr1              65706
addr2              65706
dist1             352271
dist2             552913
P_emaildomain      94456
R_emaildomain     453249
C1                     0
C2                     0
C3                     0
C4                     0
C5                     0
C6                     0
C7                     0
C8                     0
C9                     0
C10                    0
C11                    0
C12                    0
C13                    0
                   ...  
V310                  12
V311                  12
V312                  12
V313                1269
V314                1269
V315                1269
V316                  12
V317                  12
V318                  12


In [49]:
train_transaction['TransactionDT']

0            86400
1            86401
2            86469
3            86499
4            86506
5            86510
6            86522
7            86529
8            86535
9            86536
10           86549
11           86555
12           86564
13           86585
14           86596
15           86618
16           86620
17           86668
18           86725
19           86730
20           86761
21           86769
22           86786
23           86808
24           86821
25           86944
26           86945
27           86972
28           86973
29           86979
            ...   
590510    15810518
590511    15810549
590512    15810563
590513    15810575
590514    15810578
590515    15810589
590516    15810592
590517    15810626
590518    15810760
590519    15810774
590520    15810785
590521    15810802
590522    15810823
590523    15810833
590524    15810836
590525    15810866
590526    15810876
590527    15810883
590528    15810907
590529    15810912
590530    15810926
590531    15

In [58]:
train_data

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for i in train_data.columns:
    string=i
    print(string,"=={}".format(train_data[i].isnull().sum()))

TransactionID ==0
isFraud ==0
TransactionDT ==0
TransactionAmt ==0
ProductCD ==0
card1 ==0
card2 ==8933
card3 ==1565
card4 ==1577
card5 ==4259
card6 ==1571
addr1 ==65706
addr2 ==65706
dist1 ==352271
dist2 ==552913
P_emaildomain ==94456
R_emaildomain ==453249
C1 ==0
C2 ==0
C3 ==0
C4 ==0
C5 ==0
C6 ==0
C7 ==0
C8 ==0
C9 ==0
C10 ==0
C11 ==0
C12 ==0
C13 ==0
C14 ==0
D1 ==1269
D2 ==280797
D3 ==262878
D4 ==168922
D5 ==309841
D6 ==517353
D7 ==551623
D8 ==515614
D9 ==515614
D10 ==76022
D11 ==279287
D12 ==525823
D13 ==528588
D14 ==528353
D15 ==89113
M1 ==271100
M2 ==271100
M3 ==271100
M4 ==281444
M5 ==350482
M6 ==169360
M7 ==346265
M8 ==346252
M9 ==346252
V1 ==279287
V2 ==279287
V3 ==279287
V4 ==279287
V5 ==279287
V6 ==279287
V7 ==279287
V8 ==279287
V9 ==279287
V10 ==279287
V11 ==279287
V12 ==76073
V13 ==76073
V14 ==76073
V15 ==76073
V16 ==76073
V17 ==76073
V18 ==76073
V19 ==76073
V20 ==76073
V21 ==76073
V22 ==76073
V23 ==76073
V24 ==76073
V25 ==76073
V26 ==76073
V27 ==76073
V28 ==76073
V29 ==7607

In [71]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

In [73]:
train_data=train_data.drop(['isFraud'],axis=1)

In [74]:
ytrain=train_transaction['isFraud']

In [6]:
cols=['TransactionDT','TransactionAmt','ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_emaildomain',
      'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','D1',
      'V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22',
     'V23','V24','V25','V26','V27','V28','V29','V30','V31','V32','V33','V34','V95','V96'
     ,'V97','V98','V99','V100','V101','V102','V103','V104','V105','V106','V107','V108','V109','V110',
     'V111','V112','V113','V114','V115','V116','V117','V118','V119','V120','V121','V122','V123'
     ,'V124','V125','V126','V127','V128','V129','V130','V131','V132','V133','V134','V135','V136','V137',
     'V279','V280','V281','V282','V283','V284','V285','V286','V287','V288','V289','V290','V291','V292','V293'
     ,'V294','V295','V296','V297','V298','V299','V300','V301','V302','V303','V304','V305','V306','V307'
     ,'V308','V309','V310','V311','V312','V313','V314','V315','V316','V317','V318','V319','V320','V321']

In [7]:
new_train=pd.DataFrame([])

In [9]:
for i in cols:
    new_train[i]=train_data[i]

In [10]:
new_train

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,86400,68.500,W,13926,NaN,150.0,discover,142.0,credit,315.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,117.000000,0.000000,0.000000,0.000000,0.000000
1,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,86469,59.000,W,4663,490.0,150.0,visa,166.0,debit,330.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,...,135.000000,0.000000,0.000000,0.000000,50.000000,1404.000000,790.000000,0.000000,0.000000,0.000000
4,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,86510,49.000,W,5937,555.0,150.0,visa,226.0,debit,272.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,86522,159.000,W,12308,360.0,150.0,visa,166.0,debit,126.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,86529,422.500,W,12695,490.0,150.0,visa,226.0,debit,325.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,86535,15.000,H,2803,100.0,150.0,visa,226.0,debit,337.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,debit,204.0,...,0.000000,0.000000,495.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
import missingno
missingno.matrix(new_train,figsize=(12,5))

MemoryError: 

In [200]:
train_data['DeviceType'].value_counts()

desktop    85165
mobile     55645
Name: DeviceType, dtype: int64

In [204]:
display(train_data)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
for i in cols:
    print(new_train[i].dtypes)

int64
float64
object
int64
float64
float64
object
float64
object
float64
float64
object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


In [76]:
X_train, X_test, y_train, y_test = train_test_split(train_data,ytrain, test_size=0.4, random_state=0)

In [77]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [78]:
model_xgb.fit(X_train, y_train)
prediction_xgb = model_xgb.predict(X_test)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields ProductCD, card4, card6, P_emaildomain, R_emaildomain, M1, M2, M3, M4, M5, M6, M7, M8, M9, id_12, id_15, id_16, id_23, id_27, id_28, id_29, id_30, id_31, id_33, id_34, id_35, id_36, id_37, id_38, DeviceType, DeviceInfo